In [59]:
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [61]:
train_dir = 'path_to_train_images'
val_dir = 'path_to_validation_images'

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [62]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


image_size = (300, 300)
batch_size = 32

train_dir = "potato/Training"
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=image_size,
                                                    batch_size=batch_size, 
                                                    class_mode='categorical')

test_dir = "potato/Testing"
test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                  target_size=image_size,
                                                  batch_size=batch_size, 
                                                  class_mode='categorical')

Found 3251 images belonging to 3 classes.
Found 405 images belonging to 3 classes.


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def get_model():
    model = Sequential()

    # 1st layer CNN
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(300,300,3)))
    model.add(MaxPooling2D(pool_size=(5,5)))

    # 2nd layer CNN
    model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5)))

    # 3rd layer CNN
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=2))

    # 4th layer CNN
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))

    # Add flatten layer
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))

    
    model.add(Dense(3, activation='softmax'))  

    return model

In [23]:
model = get_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 298, 298, 128)     3584      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 59, 59, 128)      0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 57, 57, 256)       295168    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 11, 11, 256)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 9, 9, 512)         1180160   
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 4, 512)       

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [27]:

history = model.fit(train_generator, epochs=10, validation_data = test_generator)

Epoch 1/10
102/102 [==============================] - 489s 5s/step - loss: 0.9495 - accuracy: 0.5451 - val_loss: 0.7383 - val_accuracy: 0.7086
Epoch 2/10
102/102 [==============================] - 641s 6s/step - loss: 0.7697 - accuracy: 0.6718 - val_loss: 0.6115 - val_accuracy: 0.7951
Epoch 3/10
102/102 [==============================] - 520s 5s/step - loss: 0.4275 - accuracy: 0.8364 - val_loss: 0.5432 - val_accuracy: 0.8173
Epoch 4/10
102/102 [==============================] - 1000s 10s/step - loss: 0.2954 - accuracy: 0.8871 - val_loss: 0.2293 - val_accuracy: 0.9235
Epoch 5/10
102/102 [==============================] - 516s 5s/step - loss: 0.1907 - accuracy: 0.9326 - val_loss: 0.2546 - val_accuracy: 0.9037
Epoch 6/10
102/102 [==============================] - 443s 4s/step - loss: 0.1766 - accuracy: 0.9360 - val_loss: 0.1802 - val_accuracy: 0.9383
Epoch 7/10
102/102 [==============================] - 552s 5s/step - loss: 0.1378 - accuracy: 0.9523 - val_loss: 0.1998 - val_accuracy: 0.95

In [63]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import LearningRateScheduler


In [72]:
input_shape = (150, 150, 3)
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
model_1 = build_model()

def build_model():
    model = Sequential([
        # Convolutional Layer to handle 3D input for images
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),  # Flatten before passing to Dense layers
        
        # Dense layers for classification
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),  # L2 regularization
        layers.Dropout(0.5),  # Dropout to avoid overfitting
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001)),  # L1 regularization
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # For binary classification
    ])

    optimizer = optimizers.Adam(learning_rate=0.001)  # Adam optimizer with learning rate

    model.compile(optimizer=optimizer, 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

    return model


In [75]:
# Then fit the model with your generators:
history = model_1.fit(train_generator, 
                      steps_per_epoch=train_generator.samples // train_generator.batch_size, 
                      validation_data=test_generator, 
                      validation_steps=test_generator.samples // test_generator.batch_size, 
                      epochs=10)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_4/flatten_2/Reshape' defined at (most recent call last):
    File "c:\Users\St Prince N\anaconda3\envs\Crop-Disease-Detection\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\St Prince N\anaconda3\envs\Crop-Disease-Detection\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\St Prince N\anaconda3\envs\Crop-Disease-Detection\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\St Prince N\anaconda3\envs\Crop-Disease-Detection\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\St Prince N\anaconda3\envs\Crop-Disease-Detection\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\St Prince N\anaconda3\envs\Crop-Disease-Detection\lib\si